# Ford GoBike / Lyft Bay Wheels
- This notebook imports data from the [archives](https://s3.amazonaws.com/baywheels-data/index.html) linked to from the [System Data](https://www.fordgobike.com/system-data) page of the Lyft Bikes web site
- Data is prepared inserted into a SQL Lite database
- Exploratory data analysis and cleaning, and explanatory data analysis are conducted in subsequent notebooks

### According to the System Data page the archives have the following structure
<br>
Each trip is anonymized and includes:

- Trip Duration (seconds)
- Start Time and Date
- End Time and Date
- Start Station ID
- Start Station Name
- Start Station Latitude
- Start Station Longitude
- End Station ID
- End Station Name
- End Station Latitude
- End Station Longitude
- Bike ID
- User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path

In [2]:
import time
import sqlite3

In [3]:
from sqlalchemy import MetaData
from sqlalchemy import create_engine
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String, Float

In [4]:
DATA = Path('data/')
DATA  

WindowsPath('data')

In [5]:
BWDATA = Path('data_baywheels/')
BWDATA  

WindowsPath('data_baywheels')

# Data Import

## Ford GoBike data

### A .csv file and zip files are saved to the data directory

In [6]:
df_2017 = pd.read_csv(DATA/'2017-fordgobike-tripdata.csv')

In [7]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519700 entries, 0 to 519699
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             519700 non-null  int64  
 1   start_time               519700 non-null  object 
 2   end_time                 519700 non-null  object 
 3   start_station_id         519700 non-null  int64  
 4   start_station_name       519700 non-null  object 
 5   start_station_latitude   519700 non-null  float64
 6   start_station_longitude  519700 non-null  float64
 7   end_station_id           519700 non-null  int64  
 8   end_station_name         519700 non-null  object 
 9   end_station_latitude     519700 non-null  float64
 10  end_station_longitude    519700 non-null  float64
 11  bike_id                  519700 non-null  int64  
 12  user_type                519700 non-null  object 
 13  member_birth_year        453159 non-null  float64
 14  memb

In [8]:
files = list(x for x in DATA.iterdir() if x.is_file())

In [9]:
files

[WindowsPath('data/2017-fordgobike-tripdata.csv'),
 WindowsPath('data/201801-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201802-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201803-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201804-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201805-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201806-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201807-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201808-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201809-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201810-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201811-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201812-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201901-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201902-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201903-fordgobike-tripdata.csv.zip'),
 WindowsPath('data/201904-fordgobike-tripdata.csv.zip')]

### Check all if .csv column names and file structures are the same
- list the columns and save the dataframes into a list...

In [10]:
%%time

df_list = []
for file in files:
    n_d = file.name.index("-")
    print(file.name)
    if file.name[-3:] == "zip":
        df_bikedata = pd.read_csv(file, compression='zip')
    else:
        df_bikedata = pd.read_csv(file)

    df_cols = list(df_bikedata.columns)
    print(df_cols)
    df_list.append([file.name[:n_d], df_bikedata])

2017-fordgobike-tripdata.csv
['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'member_birth_year', 'member_gender']
201801-fordgobike-tripdata.csv.zip
['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'member_birth_year', 'member_gender', 'bike_share_for_all_trip']
201802-fordgobike-tripdata.csv.zip
['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'member_birth_year', 'member_gender', 'bike_share_f

In [11]:
len(df_list)

17

### The `2017` file differs from subsequent files
- Compare, use the largest structure as the common structure
- Which is the structure of the latest file

In [12]:
print("time period: {}".format(df_list[0:][0][0]))
print(df_list[0:][0][1].info())

time period: 2017
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519700 entries, 0 to 519699
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             519700 non-null  int64  
 1   start_time               519700 non-null  object 
 2   end_time                 519700 non-null  object 
 3   start_station_id         519700 non-null  int64  
 4   start_station_name       519700 non-null  object 
 5   start_station_latitude   519700 non-null  float64
 6   start_station_longitude  519700 non-null  float64
 7   end_station_id           519700 non-null  int64  
 8   end_station_name         519700 non-null  object 
 9   end_station_latitude     519700 non-null  float64
 10  end_station_longitude    519700 non-null  float64
 11  bike_id                  519700 non-null  int64  
 12  user_type                519700 non-null  object 
 13  member_birth_year        453159 non-null 

In [13]:
print("time period: {}".format(df_list[-1:][0][0]))
print(df_list[-1:][0][1].info())

time period: 201904
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239111 entries, 0 to 239110
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             239111 non-null  int64  
 1   start_time               239111 non-null  object 
 2   end_time                 239111 non-null  object 
 3   start_station_id         239047 non-null  float64
 4   start_station_name       239047 non-null  object 
 5   start_station_latitude   239111 non-null  float64
 6   start_station_longitude  239111 non-null  float64
 7   end_station_id           239047 non-null  float64
 8   end_station_name         239047 non-null  object 
 9   end_station_latitude     239111 non-null  float64
 10  end_station_longitude    239111 non-null  float64
 11  bike_id                  239111 non-null  int64  
 12  user_type                239111 non-null  object 
 13  member_birth_year        227912 non-nul

### The `start_station_id` and `end_station_id` are `floats` here 
- See data exploration for more detailed understanding
- They are floats because there are `NaNs` in the data, but the underlying data type should be an `int` 

# Bay Wheels data
- This is new data, after Lyft took over the FordGo business, and renamed it Bay Wheels
- I have saved it to a separate folder

In [14]:
bwfiles = list(x for x in BWDATA.iterdir() if x.is_file())

In [15]:
%%time

df_bwlist = []
for file in bwfiles:
    n_d = file.name.index("-")
    print(file.name)
    if file.name[-3:] == "zip":
        zip_file = ZipFile(file)
        # Don't import names starting with '_' - i.e. ignore the _MACOSX folder and files
        dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename), low_memory=False)
               for text_file in zip_file.infolist()
                   if ((text_file.filename.endswith('.csv')) and not ((text_file.filename.startswith('_'))))}
        df_bikedata = pd.DataFrame(*dfs.values())
    else:
        df_bikedata = pd.read_csv(file)

    df_cols = list(df_bikedata.columns)
    print(df_cols)
    df_bwlist.append([file.name[:n_d], df_bikedata])

201905-baywheels-tripdata.csv.zip
['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'bike_share_for_all_trip']
201906-baywheels-tripdata.csv.zip
['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'bike_share_for_all_trip', 'rental_access_method']
201907-baywheels-tripdata.csv.zip
['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'bike_share_for_all_trip', 'rental_access_method']
201908-baywheels-tripd

In [16]:
print("time period: {}".format(df_bwlist[0:][0][0]))
print(df_bwlist[0:][0][1].info())

time period: 201905
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182163 entries, 0 to 182162
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             182163 non-null  int64  
 1   start_time               182163 non-null  object 
 2   end_time                 182163 non-null  object 
 3   start_station_id         182148 non-null  float64
 4   start_station_name       182148 non-null  object 
 5   start_station_latitude   182163 non-null  float64
 6   start_station_longitude  182163 non-null  float64
 7   end_station_id           182148 non-null  float64
 8   end_station_name         182148 non-null  object 
 9   end_station_latitude     182163 non-null  float64
 10  end_station_longitude    182163 non-null  float64
 11  bike_id                  182163 non-null  int64  
 12  user_type                182163 non-null  object 
 13  bike_share_for_all_trip  182163 non-nul

In [17]:
print("time period: {}".format(df_bwlist[1:][0][0]))
print(df_bwlist[1:][0][1].info())

time period: 201906
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191772 entries, 0 to 191771
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             191772 non-null  int64  
 1   start_time               191772 non-null  object 
 2   end_time                 191772 non-null  object 
 3   start_station_id         189612 non-null  float64
 4   start_station_name       189612 non-null  object 
 5   start_station_latitude   191772 non-null  float64
 6   start_station_longitude  191772 non-null  float64
 7   end_station_id           189510 non-null  float64
 8   end_station_name         189510 non-null  object 
 9   end_station_latitude     191772 non-null  float64
 10  end_station_longitude    191772 non-null  float64
 11  bike_id                  191772 non-null  int64  
 12  user_type                191772 non-null  object 
 13  bike_share_for_all_trip  187879 non-nul

In [18]:
print("time period: {}".format(df_bwlist[10:][0][0]))
print(df_bwlist[10:][0][1].info())

time period: 202003
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176799 entries, 0 to 176798
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             176799 non-null  int64  
 1   start_time               176799 non-null  object 
 2   end_time                 176799 non-null  object 
 3   start_station_id         110696 non-null  float64
 4   start_station_name       111349 non-null  object 
 5   start_station_latitude   176799 non-null  float64
 6   start_station_longitude  176799 non-null  float64
 7   end_station_id           111097 non-null  float64
 8   end_station_name         111794 non-null  object 
 9   end_station_latitude     176799 non-null  float64
 10  end_station_longitude    176799 non-null  float64
 11  bike_id                  176799 non-null  int64  
 12  user_type                176799 non-null  object 
 13  rental_access_method     114269 non-nul

In [19]:
print("time period: {}".format(df_bwlist[-1:][0][0]))
print(df_bwlist[-1:][0][1].info())

time period: 202005
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138251 entries, 0 to 138250
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             138251 non-null  object 
 1   rideable_type       138251 non-null  object 
 2   started_at          138251 non-null  object 
 3   ended_at            138251 non-null  object 
 4   start_station_name  87315 non-null   object 
 5   start_station_id    87315 non-null   float64
 6   end_station_name    84779 non-null   object 
 7   end_station_id      84779 non-null   float64
 8   start_lat           138251 non-null  float64
 9   start_lng           138251 non-null  float64
 10  end_lat             138027 non-null  float64
 11  end_lng             138027 non-null  float64
 12  member_casual       138251 non-null  object 
 13  is_equity           138251 non-null  bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 13.8+ MB
None


### The last 2 dataframes of the Bay Wheels data are completely different, so exclude them...

In [20]:
len(df_bwlist)

13

In [21]:
df_bwlist = df_bwlist[:-2]

In [22]:
len(df_bwlist)

11

# Import into a SQL database

## Prepare data for import

### Convert date texts to datetime

In [23]:
for tablerow in df_list:
    print(tablerow[0])
    tablerow[1]['start_time'] = pd.to_datetime(tablerow[1]['start_time'])
    tablerow[1]['end_time'] = pd.to_datetime(tablerow[1]['end_time'])

2017
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904


In [24]:
for tablerow in df_bwlist:
    print(tablerow[0])
    tablerow[1]['start_time'] = pd.to_datetime(tablerow[1]['start_time'])
    tablerow[1]['end_time'] = pd.to_datetime(tablerow[1]['end_time'])

201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003


### Make dataframes identical shape
- Allow to process into SQL database uniformly...
- Add column `rental_access_method` found in the Bay Water data
- Add columns `member_birth_year`, `member_gender` and `bike_share_for_all_trip` found in only some tables

In [25]:
for tablerow in df_list:
    print(tablerow[0])
    cols = tablerow[1].columns
    
    if not "member_birth_year" in cols:
        tablerow[1]["member_birth_year"] = np.NaN
    if not "member_gender" in cols:
        tablerow[1]["member_gender"] = np.NaN
    if not "bike_share_for_all_trip" in cols:
        tablerow[1]["bike_share_for_all_trip"] = np.NaN
    if not "rental_access_method" in cols:
        tablerow[1]["rental_access_method"] = np.NaN

    tablerow[1] = tablerow[1][["duration_sec","start_time","end_time","start_station_id","start_station_name",
                              "start_station_latitude","start_station_longitude","end_station_id","end_station_name",
                              "end_station_latitude","end_station_longitude","bike_id","user_type","member_birth_year",
                              "member_gender","bike_share_for_all_trip","rental_access_method"]].copy() 

2017
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904


In [26]:
for tablerow in df_bwlist:
    print(tablerow[0])
    cols = tablerow[1].columns
    
    if not "member_birth_year" in cols:
        tablerow[1]["member_birth_year"] = np.NaN
    if not "member_gender" in cols:
        tablerow[1]["member_gender"] = np.NaN
    if not "bike_share_for_all_trip" in cols:
        tablerow[1]["bike_share_for_all_trip"] = np.NaN
    if not "rental_access_method" in cols:
        tablerow[1]["rental_access_method"] = np.NaN

    tablerow[1] = tablerow[1][["duration_sec","start_time","end_time","start_station_id","start_station_name",
                              "start_station_latitude","start_station_longitude","end_station_id","end_station_name",
                              "end_station_latitude","end_station_longitude","bike_id","user_type","member_birth_year",
                              "member_gender","bike_share_for_all_trip","rental_access_method"]].copy() 

201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003


### Check results of prepration so far...

In [27]:
df_list[0:][0][1].head() 

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method
0,80110,2017-12-31 16:57:39.654,2018-01-01 15:12:50.245,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,1987.0,Male,NaN,NaN
1,78800,2017-12-31 15:56:34.842,2018-01-01 13:49:55.617,284,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96,Dolores St at 15th St,37.766210,-122.426614,88,Customer,1965.0,Female,NaN,NaN
2,45768,2017-12-31 22:45:48.411,2018-01-01 11:28:36.883,245,Downtown Berkeley BART,37.870348,-122.267764,245,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer,NaN,NaN,NaN,NaN
3,62172,2017-12-31 17:31:10.636,2018-01-01 10:47:23.531,60,8th St at Ringold St,37.774520,-122.409449,5,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer,NaN,NaN,NaN,NaN
4,43603,2017-12-31 14:23:14.001,2018-01-01 02:29:57.571,239,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber,1997.0,Female,NaN,NaN


In [28]:
df_list[-1:][0][1].head() 

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method
0,50305,2019-04-30 22:33:55.155,2019-05-01 12:32:20.454,368.0,Myrtle St at Polk St,37.785434,-122.419622,324.0,Union Square (Powell St at Post St),37.788300,-122.408531,2749,Subscriber,1989.0,Female,No,
1,53725,2019-04-30 20:43:41.632,2019-05-01 11:39:06.917,246.0,Berkeley Civic Center,37.869060,-122.270556,241.0,Ashby BART Station,37.852477,-122.270213,2608,Customer,NaN,NaN,No,
2,78072,2019-04-30 10:32:46.489,2019-05-01 08:13:58.975,64.0,5th St at Brannan St,37.776754,-122.399018,64.0,5th St at Brannan St,37.776754,-122.399018,258,Subscriber,1993.0,Male,No,
3,78969,2019-04-30 10:00:51.550,2019-05-01 07:57:01.262,67.0,San Francisco Caltrain Station 2 (Townsend St...,37.776639,-122.395526,89.0,Division St at Potrero Ave,37.769218,-122.407646,1974,Subscriber,1972.0,Male,No,
4,1128,2019-04-30 23:59:04.739,2019-05-01 00:17:53.091,124.0,19th St at Florida St,37.760447,-122.410807,5.0,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,877,Subscriber,1973.0,Male,No,


In [29]:
df_bwlist[0:][0][1].head() 

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method
0,48877,2019-05-31 20:34:56.811,2019-06-01 10:09:34.796,321.0,5th St at Folsom,37.780146,-122.403071,60.0,8th St at Ringold St,37.774520,-122.409449,1416,Customer,NaN,NaN,No,NaN
1,47050,2019-05-31 19:43:56.196,2019-06-01 08:48:06.919,246.0,Berkeley Civic Center,37.869060,-122.270556,266.0,Parker St at Fulton St,37.862464,-122.264791,1797,Subscriber,NaN,NaN,No,NaN
2,5912,2019-05-31 23:54:24.089,2019-06-01 01:32:56.982,149.0,Emeryville Town Hall,37.831275,-122.285633,149.0,Emeryville Town Hall,37.831275,-122.285633,3612,Customer,NaN,NaN,No,NaN
3,2629,2019-05-31 23:59:57.449,2019-06-01 00:43:46.874,186.0,Lakeside Dr at 14th St,37.801319,-122.262642,186.0,Lakeside Dr at 14th St,37.801319,-122.262642,2333,Customer,NaN,NaN,No,NaN
4,4235,2019-05-31 23:05:48.932,2019-06-01 00:16:24.457,34.0,Father Alfred E Boeddeker Park,37.783988,-122.412408,368.0,Myrtle St at Polk St,37.785434,-122.419622,181,Subscriber,NaN,NaN,No,NaN


In [30]:
df_bwlist[-1:][0][1].head() 

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method
0,35187,2020-03-31 20:42:10.079,2020-04-01 06:28:37.844,462.0,Cruise Terminal at Pier 27,37.804648,-122.402087,24.0,Spear St at Folsom St,37.789677,-122.390428,10982,Customer,NaN,NaN,NaN,NaN
1,14568,2020-03-31 22:45:25.501,2020-04-01 02:48:13.773,42.0,San Francisco City Hall (Polk St at Grove St),37.778650,-122.418230,370.0,Jones St at Post St,37.787327,-122.413278,12617,Customer,NaN,NaN,NaN,NaN
2,35990,2020-03-31 15:08:22.331,2020-04-01 01:08:12.990,391.0,1st St at Younger Ave,37.355030,-121.904436,397.0,Gish Rd at 1st St,37.361867,-121.909315,12812,Customer,NaN,NaN,NaN,NaN
3,1068,2020-03-31 23:55:00.426,2020-04-01 00:12:49.020,456.0,Arguello Blvd at Geary Blvd,37.781468,-122.458806,107.0,17th St at Dolores St,37.763015,-122.426497,12955,Customer,NaN,NaN,NaN,NaN
4,3300,2020-03-31 23:00:55.641,2020-03-31 23:55:56.611,6.0,The Embarcadero at Sansome St,37.804770,-122.403234,24.0,Spear St at Folsom St,37.789677,-122.390428,13050,Customer,NaN,NaN,NaN,NaN


### Add a column to say which file data came from

In [31]:
for tablerow in df_list:
    print(tablerow[0])
    tablerow[1]["datasource"] = tablerow[0]

2017
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904


In [32]:
for tablerow in df_bwlist:
    print(tablerow[0])
    tablerow[1]["datasource"] = tablerow[0]

201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003


In [33]:
df_list[0:][0][1].head(1) 

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method,datasource
0,80110,2017-12-31 16:57:39.654,2018-01-01 15:12:50.245,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,1987.0,Male,NaN,NaN,2017


In [34]:
df_list[-1:][0][1].head(1) 

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method,datasource
0,50305,2019-04-30 22:33:55.155,2019-05-01 12:32:20.454,368.0,Myrtle St at Polk St,37.785434,-122.419622,324.0,Union Square (Powell St at Post St),37.7883,-122.408531,2749,Subscriber,1989.0,Female,No,NaN,201904


### Add the index as a column 

In [35]:
for tablerow in df_list:
    print(tablerow[0])
    tablerow[1] = tablerow[1].reset_index()
    tablerow[1].rename(columns={"index":"rowindex"}, inplace=True)

2017
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904


In [36]:
for tablerow in df_bwlist:
    print(tablerow[0])
    tablerow[1] = tablerow[1].reset_index()
    tablerow[1].rename(columns={"index":"rowindex"}, inplace=True)

201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003


In [37]:
for tablerow in df_list:
    print(tablerow[0])
    print(tablerow[1].columns)

2017
Index(['rowindex', 'duration_sec', 'start_time', 'end_time',
       'start_station_id', 'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'member_birth_year', 'member_gender', 'bike_share_for_all_trip',
       'rental_access_method', 'datasource'],
      dtype='object')
201801
Index(['rowindex', 'duration_sec', 'start_time', 'end_time',
       'start_station_id', 'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'member_birth_year', 'member_gender', 'bike_share_for_all_trip',
       'rental_access_method', 'datasource'],
      dtype='object')
201802
Index(['rowindex', 'duration_sec', 'start_time', 'end_time',
       'start_station_id', 'start_station_name', 'start_station_

In [38]:
df_list[0:][0][1].head(1) 

,rowindex,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method,datasource
0,0,80110,2017-12-31 16:57:39.654,2018-01-01 15:12:50.245,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,1987.0,Male,,,2017


In [39]:
df_list[-1:][0][1].head(1) 

,rowindex,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method,datasource
0,0,50305,2019-04-30 22:33:55.155,2019-05-01 12:32:20.454,368.0,Myrtle St at Polk St,37.785434,-122.419622,324.0,Union Square (Powell St at Post St),37.7883,-122.408531,2749,Subscriber,1989.0,Female,No,,201904


In [40]:
df_bwlist[-1:][0][1].head(1) 

,rowindex,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,rental_access_method,datasource
0,0,35187,2020-03-31 20:42:10.079,2020-04-01 06:28:37.844,462.0,Cruise Terminal at Pier 27,37.804648,-122.402087,24.0,Spear St at Folsom St,37.789677,-122.390428,10982,Customer,0,,,NaN,202003


### Construct a table to insert into

In [41]:
pd.io.sql.get_schema(df_list[-1:][0][1], 'tripdata')

'CREATE TABLE "tripdata" (\n"rowindex" INTEGER,\n  "duration_sec" INTEGER,\n  "start_time" TIMESTAMP,\n  "end_time" TIMESTAMP,\n  "start_station_id" REAL,\n  "start_station_name" TEXT,\n  "start_station_latitude" REAL,\n  "start_station_longitude" REAL,\n  "end_station_id" REAL,\n  "end_station_name" TEXT,\n  "end_station_latitude" REAL,\n  "end_station_longitude" REAL,\n  "bike_id" INTEGER,\n  "user_type" TEXT,\n  "member_birth_year" REAL,\n  "member_gender" TEXT,\n  "bike_share_for_all_trip" TEXT,\n  "rental_access_method" REAL,\n  "datasource" TEXT\n)'

### Some data types are corrected
- Integers are being specified for the `start_station_id`, `end_station_id` and `member_birth_year`
- Text is specified for `rental_access_method` (its only been set to flat to account for `NaN`)

In [42]:
tripdata_sql = 'CREATE TABLE "tripdata" (\n  "recid" INTEGER PRIMARY KEY,\n  "rowindex" INTEGER,\n  "duration_sec" INTEGER,\n'+\
               '  "start_time" TIMESTAMP,\n  "end_time" TIMESTAMP,\n  "start_station_id" INTEGER,\n  "start_station_name" TEXT,\n'+\
               '  "start_station_latitude" REAL,\n  "start_station_longitude" REAL,\n  "end_station_id" INTEGER,\n'+\
               '  "end_station_name" TEXT,\n  "end_station_latitude" REAL,\n  "end_station_longitude" REAL,\n'+\
               '  "bike_id" INTEGER,\n  "user_type" TEXT,\n  "member_birth_year" INTEGER,\n  "member_gender" TEXT,\n'+\
               '  "bike_share_for_all_trip" TEXT,\n  "rental_access_method" TEXT,\n  "datasource" TEXT)'

In [43]:
print(tripdata_sql)

CREATE TABLE "tripdata" (
  "recid" INTEGER PRIMARY KEY,
  "rowindex" INTEGER,
  "duration_sec" INTEGER,
  "start_time" TIMESTAMP,
  "end_time" TIMESTAMP,
  "start_station_id" INTEGER,
  "start_station_name" TEXT,
  "start_station_latitude" REAL,
  "start_station_longitude" REAL,
  "end_station_id" INTEGER,
  "end_station_name" TEXT,
  "end_station_latitude" REAL,
  "end_station_longitude" REAL,
  "bike_id" INTEGER,
  "user_type" TEXT,
  "member_birth_year" INTEGER,
  "member_gender" TEXT,
  "bike_share_for_all_trip" TEXT,
  "rental_access_method" TEXT,
  "datasource" TEXT)


In [44]:
conn = sqlite3.connect('fordgobike.db')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS tripdata')
c.execute(tripdata_sql)
conn.commit()

In [45]:
engine = create_engine('sqlite:///fordgobike.db', echo=False)

In [46]:
metadata = MetaData(engine)

In [47]:
# reflect db schema to MetaData
metadata.reflect(bind=engine)
print(metadata.tables)

immutabledict({'tripdata': Table('tripdata', MetaData(bind=Engine(sqlite:///fordgobike.db)), Column('recid', INTEGER(), table=<tripdata>, primary_key=True, nullable=False), Column('rowindex', INTEGER(), table=<tripdata>), Column('duration_sec', INTEGER(), table=<tripdata>), Column('start_time', TIMESTAMP(), table=<tripdata>), Column('end_time', TIMESTAMP(), table=<tripdata>), Column('start_station_id', INTEGER(), table=<tripdata>), Column('start_station_name', TEXT(), table=<tripdata>), Column('start_station_latitude', REAL(), table=<tripdata>), Column('start_station_longitude', REAL(), table=<tripdata>), Column('end_station_id', INTEGER(), table=<tripdata>), Column('end_station_name', TEXT(), table=<tripdata>), Column('end_station_latitude', REAL(), table=<tripdata>), Column('end_station_longitude', REAL(), table=<tripdata>), Column('bike_id', INTEGER(), table=<tripdata>), Column('user_type', TEXT(), table=<tripdata>), Column('member_birth_year', INTEGER(), table=<tripdata>), Column('

In [48]:
for _t in metadata.tables:
    print("Table: ", _t)

Table:  tripdata


https://stackoverflow.com/questions/56866187/how-can-i-print-schema-table-definitions-in-sqlalchemy

In [49]:
def dump(sql, *multiparams, **params):
    print(sql.compile(dialect=engine.dialect))

print_engine = create_engine('sqlite:///fordgobike.db', strategy='mock', executor=dump)
metadata.create_all(print_engine, checkfirst=False)


CREATE TABLE tripdata (
	recid INTEGER NOT NULL, 
	rowindex INTEGER, 
	duration_sec INTEGER, 
	start_time TIMESTAMP, 
	end_time TIMESTAMP, 
	start_station_id INTEGER, 
	start_station_name TEXT, 
	start_station_latitude REAL, 
	start_station_longitude REAL, 
	end_station_id INTEGER, 
	end_station_name TEXT, 
	end_station_latitude REAL, 
	end_station_longitude REAL, 
	bike_id INTEGER, 
	user_type TEXT, 
	member_birth_year INTEGER, 
	member_gender TEXT, 
	bike_share_for_all_trip TEXT, 
	rental_access_method TEXT, 
	datasource TEXT, 
	PRIMARY KEY (recid)
)




### Import the data

In [50]:
%%time
for tablerow in df_list:
    print(tablerow[0])
    tablerow[1].to_sql('tripdata', con=engine, if_exists='append', index=False)

2017
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
Wall time: 1min 42s


In [51]:
%%time
for tablerow in df_bwlist:
    print(tablerow[0])
    tablerow[1].to_sql('tripdata', con=engine, if_exists='append', index=False)

201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
Wall time: 1min 21s


### Commit, and close the connection

In [52]:
conn.commit()

In [53]:
conn.close()

# Appendix

In [54]:
%%javascript

// Sourced from http://nbviewer.jupyter.org/gist/minrk/5d0946d39d511d9e0b5a

$("#renumber-button").parent().remove();

function renumber() {
    // renumber cells in order
    var i=1;
    IPython.notebook.get_cells().map(function (cell) {
        if (cell.cell_type == 'code') {
            // set the input prompt
            cell.set_input_prompt(i);
            // set the output prompt (in two places)
            cell.output_area.outputs.map(function (output) {
                if (output.output_type == 'execute_result') {
                    output.execution_count = i;
                    cell.element.find(".output_prompt").text('Out[' + i + ']:');
                }
            });
            i += 1;
        }
    });
}

IPython.toolbar.add_buttons_group([{
  'label'   : 'Renumber',
  'icon'    : 'fa-list-ol',
  'callback': renumber,
  'id'      : 'renumber-button'
}]);

<IPython.core.display.Javascript object>